# 🚀 Project: Deep Learning From Scratch - NumPy MNIST Classifier

**Goal:** Implement a fully functional, two-layer Neural Network using only Python and NumPy to classify the 10 digits (0-9) of the MNIST dataset.

**Objective:** Achieve $\approx 92\%$ accuracy on the test set. Mastery of this project ensures deep understanding of all core DL concepts: Forward Propagation, Cross-Entropy Loss, and Backpropagation (Gradient Descent).

---

## 📝 Implementation Stages

### 1. Data Acquisition and Preprocessing

The network expects vectorized, scaled, and correctly encoded data.

* **1.1. Data Loading:** Load the training and testing data (e.g., via `numpy.load` or a dedicated library like `python-mnist`).
* **1.2. Flattening and Reshaping:** Convert the $28 \times 28$ input images into a $784$-element feature vector. Ensure the data matrix $X$ has a shape of $(\text{features}, \text{samples})$ or $(\text{samples}, \text{features})$ and be consistent.
* **1.3. Normalization:** Scale pixel values from $[0, 255]$ to $[0, 1]$.
* **1.4. One-Hot Encoding:** Convert integer labels (0-9) into 10-dimensional vectors ($Y$).

### 2. Network Architecture & Parameter Initialization

Define the structure and starting values for the learnable parameters.

* **2.1. Define Structure:** Set the size of the Input ($\text{N}_{in} = 784$), Hidden Layer ($\text{N}_h$, e.g., 128), and Output Layer ($\text{N}_{out} = 10$).
* **2.2. Initialize $W$ and $b$:** Create two weight matrices ($W_1, W_2$) and two bias vectors ($b_1, b_2$) using small random numbers (e.g., `np.random.randn(...) * 0.01`).

### 3. Core Mathematical Functions

Implement the engine of the neural network.

* **3.1. Activation Functions:**
    * **ReLU:** $A = \max(0, Z)$. Implement its derivative, $g'(Z)$.
    * **Softmax:** Used on the output layer for multiclass probability $\hat{Y}$.
* **3.2. Forward Propagation:** Implement the function to calculate:
    $$Z^{[1]} = W^{[1]} X + b^{[1]}$$
    $$A^{[1]} = \text{ReLU}(Z^{[1]})$$
    $$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
    $$\hat{Y} = A^{[2]} = \text{Softmax}(Z^{[2]})$$
* **3.3. Loss Function:** Implement the **Cross-Entropy Loss** calculation between $Y$ (true labels) and $\hat{Y}$ (predictions).

### 4. Backpropagation and Gradient Update

This is where the learning happens—the crucial part of the project.

* **4.1. Backward Propagation:** Implement the function to calculate the gradients of the Loss with respect to all parameters ($\frac{\partial L}{\partial W^{[1]}}, \frac{\partial L}{\partial b^{[1]}}, \frac{\partial L}{\partial W^{[2]}}, \frac{\partial L}{\partial b^{[2]}}$) using the Chain Rule.
* **4.2. Parameter Update:** Update the parameters using the Gradient Descent rule with a defined $\text{learning\_rate} \ (\alpha)$:
    $$W \leftarrow W - \alpha \cdot \frac{\partial L}{\partial W}$$

### 5. Training Loop and Evaluation

The final assembly of the components into a running training script.

* **5.1. The Training Loop:** Iterate over a fixed number of **epochs**. Within each epoch, execute steps 3.2, 3.3, 4.1, and 4.2.
* **5.2. Prediction and Accuracy:** Create a function to convert the output probabilities ($\hat{Y}$) back into a class prediction and calculate the model's classification **accuracy**.
* **5.3. Tracking:** Log the loss and accuracy metrics every $X$ epochs to visualize the learning process.

---

### **Key Metrics to Track:**
* **Learning Rate** (e.g., $0.01$)
* **Number of Epochs** (e.g., 100-500)
* **Final Test Accuracy** (Target $\ge 92\%$)

In [2]:

import numpy
# Part 1 Data Loading and Preparation (using sklearn as helper)
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split # Crucial for shuffling and splitting!
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# --- 1. Load Full MNIST Data (28x28 = 784 features) ---
# X will be a NumPy array (due to astype)
# y will be a Pandas Series
mnist = fetch_openml('mnist_784', version=1, parser='auto')
X = mnist.data.astype(np.float64)  # Features (70000, 784)
y = mnist.target.astype(np.int64) # Targets (70000,) - This is a Pandas Series!

print("--- Initial Data Shapes (Full MNIST) ---")
print(f"Features (X): {X.shape}")
print(f"Targets (y): {y.shape}\n")


# --- 2. Train-Test Split with Shuffling and Stratification ---
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)
# X_train_raw/X_test_raw are NumPy arrays.
# y_train_raw/y_test_raw are Pandas Series objects.


# --- 3. Feature Scaling (Normalization) ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_test_scaled = scaler.transform(X_test_raw)


# --- 4. Target Encoding (One-Hot) ---

# *** FIX IS HERE ***
# Convert the Pandas Series (y_train_raw/y_test_raw) to a NumPy array using .to_numpy()
y_train_reshaped = y_train_raw.to_numpy().reshape(-1, 1)
y_test_reshaped = y_test_raw.to_numpy().reshape(-1, 1)
# *******************

encoder = OneHotEncoder(sparse_output=False)
encoder.fit(y_train_reshaped)

Y_train_encoded = encoder.transform(y_train_reshaped)
Y_test_encoded = encoder.transform(y_test_reshaped)


# --- 5. CRUCIAL STEP: Apply DL Convention Transpose ---
# X: (784, M) features x samples
# Y: (10, M) classes x samples

X_train_DL = X_train_scaled.T
X_test_DL = X_test_scaled.T

Y_train_DL = Y_train_encoded.T
Y_test_DL = Y_test_encoded.T


# --- Final Verification ---
print("--- Final Shapes for DL Implementation (MNIST 784) ---")
M_train = X_train_DL.shape[1]
M_test = X_test_DL.shape[1]
print(f"Total Training Samples (M): {M_train}") # Should be 59500 (70000 * 0.85)
print(f"Total Test Samples (M): {M_test}\n")   # Should be 10500 (70000 * 0.15)
print(f"X_train (Features, Samples): {X_train_DL.shape}") # Should be (784, 59500)
print(f"Y_train (Classes, Samples):  {Y_train_DL.shape}")  # Should be (10, 59500)

--- Initial Data Shapes (Full MNIST) ---
Features (X): (70000, 784)
Targets (y): (70000,)

--- Final Shapes for DL Implementation (MNIST 784) ---
Total Training Samples (M): 59500
Total Test Samples (M): 10500

X_train (Features, Samples): (784, 59500)
Y_train (Classes, Samples):  (10, 59500)


In [2]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))

PyTorch version: 2.8.0+cu128
CUDA available: True
GPU name: NVIDIA GeForce RTX 3090


In [3]:
import numpy as np
from abc import ABC, abstractmethod
from typing import Union, Optional

NDArray = np.ndarray
LEAKY_SLOPE = 0.01

class BasicComponent(ABC):
    """Abstract base class for a computational graph node with minimal caching."""

    # Stores the input to the component (A_prev or Z) for use in the backward pass.
    input_cache: Optional[NDArray] = None

    # Derivative of Loss w.r.t input (dL/dA_prev or dL/dZ) - generally not stored here
    # but passed along the backward chain.

    # Graph Pointers (Crucial for connecting the network structure)
    prev: Union['BasicComponent', None] = None
    next: Union['BasicComponent', None] = None

    @abstractmethod
    def forward(self, input_data: NDArray) -> NDArray:
        """Performs the forward pass, caches input, and returns output."""
        pass

    @abstractmethod
    def backward(self, dL_doutput: NDArray) -> NDArray:
        """
        Calculates dL/dInput and returns it to the previous layer.
        dL_doutput is the incoming derivative from the *next* component.
        """
        pass

# --- 1. Activation Component: Leaky ReLU ---
class ReLU(BasicComponent):
    """
    Leaky ReLU Activation (A = max(0, Z) or alpha * Z).
    """
    def __init__(self, prev_component: Optional[BasicComponent] = None):
        self.prev = prev_component

    def forward(self, Z: NDArray) -> NDArray:
        """A = Leaky ReLU(Z). Caches Z."""
        self.input_cache = Z # Cache Z (the input)
        A = np.where(Z > 0, Z, Z * LEAKY_SLOPE)
        return A

    def backward(self, dL_dA: NDArray) -> NDArray:
        """
        Calculates dL/dZ = dL/dA * dA/dZ and returns dL/dZ.
        """
        Z = self.input_cache

        # dA/dZ mask: 1 where Z > 0, alpha where Z <= 0
        dA_dZ = np.where(Z > 0, 1.0, LEAKY_SLOPE)

        # Element-wise multiplication
        dL_dZ = dL_dA * dA_dZ

        # NOTE: We return the derivative directly instead of storing it
        return dL_dZ

# --- 2. Output/Loss Component: Softmax + CrossEntropy ---
class SoftmaxCrossEntropy(BasicComponent):
    """
    Combines Softmax activation and Cross-Entropy Loss.
    Requires y_true for the derivative calculation.
    """
    def __init__(self, y_true: NDArray, prev_component: Optional[BasicComponent] = None):
        self.prev = prev_component
        self.y_true = y_true # True labels (10, M)
        self.Y_pred: Optional[NDArray] = None # We MUST cache Y_pred for the backward pass

    def forward(self, Z: NDArray) -> NDArray:
        """Calculates stable Softmax probabilities (Y_pred)."""
        self.input_cache = Z

        # Numerical Stability
        Z_shifted = Z - np.max(Z, axis=0, keepdims=True)
        exp_Z = np.exp(Z_shifted)

        # Softmax
        A = exp_Z / np.sum(exp_Z, axis=0, keepdims=True)
        self.Y_pred = A
        return A # Returns Y_pred

    def backward(self, dL_doutput: NDArray = None) -> NDArray:
        """
        Calculates dL/dZ = Y_pred - Y_true.
        """
        # The crucial simplified gradient for the combined component
        dL_dZ = self.Y_pred - self.y_true

        # Average over the number of samples (M) for the overall gradient
        M = self.Y_pred.shape[1]
        dL_dZ_avg = dL_dZ / M

        # NOTE: We return the derivative (dL/dZ) directly
        return dL_dZ_avg

class Linear(BasicComponent):
    """
    Linear/Affine Transformation: Z = W * A_prev + b
    Manages W, b, and their gradients (dW, db).
    """
    def __init__(self, input_size: int, output_size: int, prev_component: Optional[BasicComponent] = None):
        self.prev = prev_component

        # He Initialization
        self.W = np.random.randn(output_size, input_size) * np.sqrt(2.0/input_size)
        # b: (output_size, 1)
        self.b = np.zeros((output_size, 1))

        # Gradient Accumulators (dW and db will be accumulated in backward pass)
        self.dW = np.zeros(self.W.shape)
        self.db = np.zeros(self.b.shape)

    def forward(self, A_prev: NDArray) -> NDArray:
        """Z = W * A_prev + b. Caches A_prev."""
        self.input_cache = A_prev # A_prev has shape (input_size, M)

        # Matrix multiplication
        Z = self.W @ A_prev + self.b
        return Z

    def backward(self, dL_dZ: NDArray) -> NDArray:
        """
        Calculates dL/dW, dL/db (accumulated), and dL/dA_prev (passed back).
        dL_dZ is the incoming derivative from the next component.
        """
        A_prev = self.input_cache # (input_size, M)

        # 1. Gradient w.r.t Bias (dL/db)
        # Sum dL/dZ across all samples (axis=1).
        # ACCUMULATE: += is used because this gradient is summed over all samples in the mini-batch
        # before a single update step.
        db = np.sum(dL_dZ, axis=1, keepdims=True)
        self.db += db

        # 2. Gradient w.r.t Weights (dL/dW)
        # dL/dW = dL/dZ @ (A_prev)^T
        # ACCUMULATE: += is used for the same reason as db.
        dW = dL_dZ @ A_prev.T
        self.dW += dW

        # 3. Gradient w.r.t Input (dL/dA_prev)
        # dL/dA_prev = W^T @ dL/dZ.
        # NO ACCUMULATION: This is an intermediate error signal (not a parameter gradient)
        # and is passed directly to the previous layer.
        dL_dA_prev = self.W.T @ dL_dZ

        return dL_dA_prev

    def reset_gradients(self):
        """Resets accumulated dW and db to zero for the next mini-batch/epoch."""
        self.dW = np.zeros(self.W.shape)
        self.db = np.zeros(self.b.shape)

In [4]:
# --- Training ---
# Full Batch Training - No need to reset_gradients inside the loop if using full batch
# and not accumulating across epochs, but it's good practice.

iteration = 100
learning_rate = 0.01

# --- Setup Graph ---
linear_1 = Linear(784, 256, prev_component=None)
relu_1 = ReLU(prev_component=linear_1)
linear_2 = Linear(256, 10, prev_component=relu_1)

# Note: y_true is set once for the entire training set
softmax_1 = SoftmaxCrossEntropy(prev_component=linear_2, y_true=Y_train_DL)

# Get total test samples for accurate calculation
M_test = Y_test_DL.shape[1]

for i in range(iteration):
    # --- Forward Pass ---
    # The result of the final forward pass is stored internally in softmax_1.Y_pred
    y_pred = softmax_1.forward(
        linear_2.forward(
            relu_1.forward(
                linear_1.forward(X_train_DL)
            )
        )
    )

    # --- Backward Pass ---
    # Starts with softmax_1.backward() which returns dL/dZ_L2
    # The subsequent calls pass the derivative backward
    dL_dA1 = linear_2.backward(softmax_1.backward())
    dL_dZ1 = relu_1.backward(dL_dA1)
    linear_1.backward(dL_dZ1) # Final backward call consumes dL_dZ1

    # --- Parameter Update ---
    # The accumulated dW/db are used for the update
    linear_1.b -= learning_rate * linear_1.db
    linear_1.W -= learning_rate * linear_1.dW
    linear_2.b -= learning_rate * linear_2.db
    linear_2.W -= learning_rate * linear_2.dW

    # Reset gradients after the update (necessary if running multiple passes per epoch,
    # but still clean practice)
    linear_1.reset_gradients()
    linear_2.reset_gradients()

    # --- Accuracy Calculation (Corrected) ---
    # 1. Forward pass for TEST data (no caching, no backward)
    y_test_pred = softmax_1.forward(
        linear_2.forward(
            relu_1.forward(
                linear_1.forward(X_test_DL)
            )
        )
    )

    # 2. Get the index of the highest probability (0-9) for predicted and true
    y_test_pred_indices = np.argmax(y_test_pred, axis=0)
    y_test_true_indices = np.argmax(Y_test_DL, axis=0)

    # 3. Calculate accuracy: count matches and divide by total samples
    correct_predictions = (y_test_pred_indices == y_test_true_indices).sum()
    accuracy = correct_predictions / M_test * 100.0

    print(f"Accuracy for iteration {i}: {accuracy:.2f}%")

Accuracy for iteration 0: 15.83%
Accuracy for iteration 1: 17.86%
Accuracy for iteration 2: 20.11%
Accuracy for iteration 3: 22.31%


KeyboardInterrupt: 

In [65]:
# --- Configuration ---
# Assuming NDArray, Linear, ReLU, SoftmaxCrossEntropy, and the DL data variables
# (X_train_DL, Y_train_DL, X_test_DL, Y_test_DL) are already defined.

# --- Define Hyperparameters ---
iteration = 1000 # Increased iterations for better results
learning_rate = 0.01
M_test = X_test_DL.shape[1] # Number of test samples

# --- 1. Define 3-Layer Architecture (784 -> 256 -> 128 -> 10) ---
print("--- Creating 3-Layer Network Structure ---")
# Layer 1: Input (784) -> Hidden (256)
linear_1 = Linear(784, 256, prev_component=None)
relu_1 = ReLU(prev_component=linear_1)

# Layer 2: Hidden (256) -> Hidden (128)
linear_2 = Linear(256, 128, prev_component=relu_1)
relu_2 = ReLU(prev_component=linear_2)

# Layer 3: Hidden (128) -> Output (10)
linear_3 = Linear(128, 10, prev_component=relu_2)
softmax_1 = SoftmaxCrossEntropy(prev_component=linear_3, y_true=Y_train_DL)

# List of all trainable components for easy iteration
trainable_components = [linear_1, linear_2, linear_3]
print(f"Network defined: {linear_1.__class__.__name__} -> ... -> {linear_3.__class__.__name__} -> Softmax")

# --- 2. Training Loop ---
for i in range(iteration):
    # 2.1. Reset Gradients for the new epoch/batch
    for comp in trainable_components:
        comp.reset_gradients()

    # 2.2. Forward Pass (Training Data)
    z1 = linear_1.forward(X_train_DL)
    a1 = relu_1.forward(z1)
    z2 = linear_2.forward(a1)
    a2 = relu_2.forward(z2)      # New Layer
    z3 = linear_3.forward(a2)    # New Layer
    y_pred = softmax_1.forward(z3)

    # 2.3. Backward Pass (Reverse Chain)
    dL_dZ3 = softmax_1.backward()
    dL_dA2 = linear_3.backward(dL_dZ3)
    dL_dZ2 = relu_2.backward(dL_dA2)        # New Layer
    dL_dA1 = linear_2.backward(dL_dZ2)
    dL_dZ1 = relu_1.backward(dL_dA1)
    linear_1.backward(dL_dZ1)

    # 2.4. Parameter Update (Gradient Descent)
    for comp in trainable_components:
        comp.b -= learning_rate * comp.db
        comp.W -= learning_rate * comp.dW

    # 2.5. Accuracy Calculation (Test Data)
    # Forward Pass on TEST data (must replicate the full chain for evaluation)
    test_z1 = linear_1.forward(X_test_DL)
    test_a1 = relu_1.forward(test_z1)
    test_z2 = linear_2.forward(test_a1)
    test_a2 = relu_2.forward(test_z2)
    test_z3 = linear_3.forward(test_a2)
    y_test_pred = softmax_1.forward(test_z3)

    # Calculate Accuracy
    y_test_pred_indices = np.argmax(y_test_pred, axis=0)
    y_test_true_indices = np.argmax(Y_test_DL, axis=0)
    correct_predictions = (y_test_pred_indices == y_test_true_indices).sum()
    accuracy = correct_predictions / M_test * 100.0

    if i % 50 == 0:
        print(f"Accuracy for iteration {i}: {accuracy:.2f}%")

print(f"\nFinal Accuracy after {iteration} iterations: {accuracy:.2f}%")

--- Creating 3-Layer Network Structure ---
Network defined: Linear -> ... -> Linear -> Softmax
Accuracy for iteration 0: 10.10%
Accuracy for iteration 50: 66.89%
Accuracy for iteration 100: 78.00%
Accuracy for iteration 150: 82.55%
Accuracy for iteration 200: 84.90%
Accuracy for iteration 250: 86.42%
Accuracy for iteration 300: 87.27%
Accuracy for iteration 350: 87.99%
Accuracy for iteration 400: 88.73%
Accuracy for iteration 450: 89.30%
Accuracy for iteration 500: 89.55%
Accuracy for iteration 550: 89.87%
Accuracy for iteration 600: 90.24%
Accuracy for iteration 650: 90.57%
Accuracy for iteration 700: 90.88%
Accuracy for iteration 750: 91.13%
Accuracy for iteration 800: 91.39%
Accuracy for iteration 850: 91.56%
Accuracy for iteration 900: 91.70%
Accuracy for iteration 950: 91.78%

Final Accuracy after 1000 iterations: 91.90%


# GPU Implementation with torch library

In [2]:
import torch
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- 0. Define Device ---
# Check if CUDA is available and set the device object accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"--- Using device: {device} ---")

# --- 1. Data Loading and Splitting (CPU/NumPy Stage) ---

# Load Full MNIST Data (70000 samples, 784 features)
# parser='auto' is required for modern sklearn versions
mnist = fetch_openml('mnist_784', version=1, parser='auto')
X = mnist.data.astype(np.float32) # Use float32 to match torch.float default
y = mnist.target.astype(np.int64)

# Train-Test Split (15% test set)
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

# Feature Scaling (Normalization) - Keeps data in NumPy/CPU
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_test_scaled = scaler.transform(X_test_raw)

# --- 2. Conversion and Transfer (CPU/NumPy -> GPU/PyTorch) ---

# PyTorch tensors for features must be float
# The shape is (M, 784) - PyTorch prefers (Samples, Features) for Linear layers
X_train_DL_GPU = torch.from_numpy(X_train_scaled).float().to(device)
X_test_DL_GPU = torch.from_numpy(X_test_scaled).float().to(device)

# Labels (Y) must be LongTensors of class indices (0-9) for CrossEntropyLoss
# y_train_raw/y_test_raw are already simple index arrays from fetch_openml
# We cast them to int64 (NumPy) then to LongTensor (PyTorch)
Y_train_DL_GPU = torch.from_numpy(y_train_raw.to_numpy()).long().to(device)
Y_test_DL_GPU = torch.from_numpy(y_test_raw.to_numpy()).long().to(device)

# --- 3. Verification ---
print("\n--- Final GPU Tensor Check ---")
print(f"X_train shape: {X_train_DL_GPU.shape}, Device: {X_train_DL_GPU.device}, Dtype: {X_train_DL_GPU.dtype}")
print(f"Y_train shape: {Y_train_DL_GPU.shape}, Device: {Y_train_DL_GPU.device}, Dtype: {Y_train_DL_GPU.dtype}")

--- Using device: cuda:0 ---

--- Final GPU Tensor Check ---
X_train shape: torch.Size([59500, 784]), Device: cuda:0, Dtype: torch.float32
Y_train shape: torch.Size([59500]), Device: cuda:0, Dtype: torch.int64


In [11]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np

# Assuming device is defined (e.g., torch.device("cuda:0"))
# Assuming X_train_DL_GPU and Y_train_DL_GPU are your GPU Tensors

# --- 1. Corrected Model Definition (3-Layer Architecture) ---
class NeuralNetwork(nn.Module):
    # We are using a 3-layer architecture (784 -> 256 -> 128 -> 10)
    # The output of the final Linear layer is the raw scores (logits)
    def __init__(self):
        super().__init__() # CRITICAL FIX: Correctly call the parent constructor
        self.architecture = nn.Sequential(
            # Layer 1
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            # Layer 2
            nn.Linear(256, 128), # Added a 128-neuron hidden layer
            nn.ReLU(),
            nn.Dropout(p=0.5),
            # Layer 3 (Output)
            nn.Linear(128, 10)   # Output layer produces 10 logits (raw scores)
            # Softmax is NOT included here, as CrossEntropyLoss handles it internally
        )

    def forward(self, x):
        return self.architecture(x)

# --- 2. Training Setup ---
epoch = 1000
learning_rate = 0.1
ADAM_LR = 0.001

model = NeuralNetwork().to(device) # Move the entire model to GPU
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=learning_rate) # Use PyTorch's SGD optimizer
optimizer = optim.Adam(model.parameters(), lr=ADAM_LR)

# --- 3. Training Loop with Correct Accuracy Calculation ---
for i in range(epoch):
    # Set the model to training mode (important for later layers like Dropout/BatchNorm)
    model.train()

    # 1. Zero Gradients
    optimizer.zero_grad()

    # 2. Forward Pass (on GPU data)
    logits = model(X_train_DL_GPU)

    # 3. Compute Loss
    # Criterion expects (Logits, True Indices)
    loss = criterion(logits, Y_train_DL_GPU)

    # 4. Backward Pass (Autograd calculates all gradients)
    loss.backward()

    # 5. Optimizer Step (Updates all parameters using the computed gradients and LR)
    optimizer.step()

    # --- ACCURACY CALCULATION (The Answer to your question) ---
    model.eval() # Set the model to evaluation mode (e.g., disables dropout)

    with torch.no_grad():

        # 1. TRAIN Accuracy (Your current calculation)
        logits_train = model(X_train_DL_GPU)
        _, predicted_train = torch.max(logits_train.data, 1)
        train_accuracy = 100 * (predicted_train == Y_train_DL_GPU).sum().item() / Y_train_DL_GPU.size(0)

        # 2. TEST Accuracy (The critical new metric)
        logits_test = model(X_test_DL_GPU)
        _, predicted_test = torch.max(logits_test.data, 1)
        test_accuracy = 100 * (predicted_test == Y_test_DL_GPU).sum().item() / Y_test_DL_GPU.size(0)

    # Set model back to train mode
    model.train()

    # Printing the key metrics
    print(f"Epoch {i+1:03d} | Loss: {loss.item():.4f} | Train Acc: {train_accuracy:.2f}% | Test Acc: {test_accuracy:.2f}%")

print(f"\nTraining Complete. Final Accuracy: {accuracy:.2f}%")

Epoch 001 | Loss: 2.3212 | Train Acc: 34.96% | Test Acc: 35.60%
Epoch 002 | Loss: 2.2243 | Train Acc: 59.98% | Test Acc: 60.19%
Epoch 003 | Loss: 2.1322 | Train Acc: 68.82% | Test Acc: 69.06%
Epoch 004 | Loss: 2.0302 | Train Acc: 71.81% | Test Acc: 71.90%
Epoch 005 | Loss: 1.9174 | Train Acc: 72.43% | Test Acc: 72.47%
Epoch 006 | Loss: 1.7967 | Train Acc: 72.08% | Test Acc: 72.13%
Epoch 007 | Loss: 1.6685 | Train Acc: 71.85% | Test Acc: 71.72%
Epoch 008 | Loss: 1.5341 | Train Acc: 71.98% | Test Acc: 71.80%
Epoch 009 | Loss: 1.4073 | Train Acc: 72.45% | Test Acc: 72.17%
Epoch 010 | Loss: 1.2877 | Train Acc: 73.46% | Test Acc: 73.11%
Epoch 011 | Loss: 1.1752 | Train Acc: 74.84% | Test Acc: 74.84%
Epoch 012 | Loss: 1.0780 | Train Acc: 76.56% | Test Acc: 76.51%
Epoch 013 | Loss: 0.9899 | Train Acc: 78.21% | Test Acc: 78.00%
Epoch 014 | Loss: 0.9133 | Train Acc: 79.80% | Test Acc: 79.52%
Epoch 015 | Loss: 0.8470 | Train Acc: 81.27% | Test Acc: 81.09%
Epoch 016 | Loss: 0.7938 | Train Acc: 82

In [40]:
z @ z.T

array([[ 2.03724074,  1.01862037,  0.        , -0.02037037, -0.04074074],
       [ 1.01862037,  0.50931019,  0.        , -0.01018519, -0.02037037],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.02037037, -0.01018519,  0.        ,  0.50931019,  1.01862037],
       [-0.04074074, -0.02037037,  0.        ,  1.01862037,  2.03724074]])

In [57]:
(z == z.max(axis=0, keepdims=True)).sum()/z.shape[1]

1.0

In [53]:
z

array([[ 1.00000000e+00,  7.77777778e-01,  5.55555556e-01,
         3.33333333e-01,  1.11111111e-01, -1.11111111e-03,
        -3.33333333e-03, -5.55555556e-03, -7.77777778e-03,
        -1.00000000e-02],
       [ 5.00000000e-01,  3.88888889e-01,  2.77777778e-01,
         1.66666667e-01,  5.55555556e-02, -5.55555556e-04,
        -1.66666667e-03, -2.77777778e-03, -3.88888889e-03,
        -5.00000000e-03],
       [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-5.00000000e-03, -3.88888889e-03, -2.77777778e-03,
        -1.66666667e-03, -5.55555556e-04,  5.55555556e-02,
         1.66666667e-01,  2.77777778e-01,  3.88888889e-01,
         5.00000000e-01],
       [-1.00000000e-02, -7.77777778e-03, -5.55555556e-03,
        -3.33333333e-03, -1.11111111e-03,  1.11111111e-01,
         3.33333333e-01,  5.55555556e-01,  7.77777778e-01,
         1.

In [36]:
numpy.exp(z)

array([[2.71828183, 2.17662993, 1.742909  , 1.39561243, 1.11751907,
        0.99888951, 0.99667222, 0.99445985, 0.99225239, 0.99004983],
       [1.64872127, 1.47534062, 1.32019279, 1.18136041, 1.05712774,
        0.9994446 , 0.99833472, 0.99722608, 0.99611866, 0.99501248],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [0.99501248, 0.99611866, 0.99722608, 0.99833472, 0.9994446 ,
        1.05712774, 1.18136041, 1.32019279, 1.47534062, 1.64872127],
       [0.99004983, 0.99225239, 0.99445985, 0.99667222, 0.99888951,
        1.11751907, 1.39561243, 1.742909  , 2.17662993, 2.71828183]])